In [ ]:
import re
import collections

def get_word_frequencies(sentences):
    """
    Preprocesses sentences and returns a frequency dictionary of words.
    Lowercase, remove periods, and add </w> to each word.
    """
    word_freq = collections.defaultdict(int)
    for sentence in sentences:
        # Lowercase, remove period, and split into words
        words = sentence.lower().replace('.', '').split()
        for word in words:
            # Append end-of-word token and count frequency
            word_freq[word + '</w>'] += 1
    return word_freq

def initialize_corpus(word_freq):
    """
    Splits each word into characters, creating the initial corpus representation.
    Example: {'the</w>': 8} -> {'t h e </w>': 8}
    """
    corpus = {}
    for word, freq in word_freq.items():
        corpus[" ".join(list(word))] = freq
    return corpus

def initialize_vocab(corpus):
    """Initializes the vocabulary with all unique characters."""
    vocab = set()
    for word in corpus:
        vocab.update(word.split())
    return list(vocab)

def get_pair_stats(corpus):
    """Counts the frequency of each adjacent pair of symbols."""
    pairs = collections.defaultdict(int)
    for word, freq in corpus.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            # Add the frequency of the word to the pair's count
            pairs[symbols[i], symbols[i+1]] += freq
    return pairs

def merge_pair(best_pair, corpus):
    """Merges the most frequent pair in the corpus."""
    new_corpus = {}
    # Create the merged token (e.g., ('e', '</w>') -> 'e</w>')
    new_token = "".join(best_pair)
    # Use regex to replace the pair in the keys of the corpus dictionary
    # We escape special characters and handle the space between tokens
    pattern = re.compile(r'(?<!\S)' + re.escape(' '.join(best_pair)) + r'(?!\S)')
    
    for word, freq in corpus.items():
        new_word = pattern.sub(new_token, word)
        new_corpus[new_word] = freq
        
    return new_corpus

def tokenize_word(input_word, vocab):
    """
    Tokenizes a word using the greedy longest-match algorithm.
    """
    # Ensure the word has the end-of-word token
    if not input_word.endswith('</w>'):
        input_word += '</w>'

    tokens = []
    current_pos = 0
    while current_pos < len(input_word):
        # Find the longest subword in the vocab that is a prefix
        best_match = ""
        for i in range(current_pos, len(input_word)):
            subword = input_word[current_pos : i+1]
            if subword in vocab:
                best_match = subword
        
        # If no match is found (should only happen for unknown chars),
        # treat the single character as the token.
        if not best_match:
            best_match = input_word[current_pos]

        tokens.append(best_match)
        current_pos += len(best_match)
        
    return tokens

# --- Main execution ---
if __name__ == "__main__":
    # 1. Dataset Initialization
    dataset = [
        "The boy hugs the cat.",
        "The boys are hugging the dogs.",
        "The dogs are chasing the cats.",
        "The dog and the cat sit quietly.",
        "The boy is sitting on the dog."
    ]
    num_merges = 20

    # 2. Preprocessing
    word_frequencies = get_word_frequencies(dataset)
    corpus = initialize_corpus(word_frequencies)
    vocab = initialize_vocab(corpus)

    print("--- Initial State ---")
    print(f"Initial Vocabulary Size: {len(vocab)}")
    # print(f"Initial Vocabulary: {sorted(vocab)}")
    # print(f"Initial Corpus: {corpus}")
    print("-" * 20)

    # 3. Training Loop (Applying Merges)
    for i in range(num_merges):
        pair_stats = get_pair_stats(corpus)
        if not pair_stats:
            break # No more pairs to merge
        
        # Find the most frequent pair
        # The key for max is a lambda function to look up the frequency in pair_stats
        # Tie-breaking is handled implicitly by Python's default behavior for max()
        best_pair = max(pair_stats, key=pair_stats.get)
        
        # Perform the merge
        corpus = merge_pair(best_pair, corpus)
        
        # Add the new merged token to the vocabulary
        new_token = "".join(best_pair)
        vocab.append(new_token)
        
        print(f"Iteration {i+1}: Merged {best_pair} -> '{new_token}' (Frequency: {pair_stats[best_pair]})")

    # 4. Final Vocabulary
    print("\n--- Final Vocabulary ---")
    print(f"Final Vocabulary Size: {len(vocab)}")
    print(sorted(vocab))
    print("-" * 20)

    # 5. Tokenize a new sentence
    new_sentence = "The cat is chasing the dog quietly."
    print(f"\n--- Tokenizing New Sentence ---\n'{new_sentence}'\n")

    tokenized_sentence = []
    for word in new_sentence.lower().replace('.', '').split():
        tokens = tokenize_word(word, vocab)
        tokenized_sentence.append(tokens)
        print(f"'{word}' -> {tokens}")
    
    print("\nFinal Tokenized Output:", tokenized_sentence)

--- Initial State ---
Initial Vocabulary Size: 21
--------------------
Iteration 1: Merged ('<', '/') -> '</' (Frequency: 31)
Iteration 2: Merged ('</', 'w') -> '</w' (Frequency: 31)
Iteration 3: Merged ('</w', '>') -> '</w>' (Frequency: 31)
Iteration 4: Merged ('e', '</w>') -> 'e</w>' (Frequency: 12)
Iteration 5: Merged ('t', 'h') -> 'th' (Frequency: 10)
Iteration 6: Merged ('th', 'e</w>') -> 'the</w>' (Frequency: 10)
Iteration 7: Merged ('s', '</w>') -> 's</w>' (Frequency: 6)
Iteration 8: Merged ('g', '</w>') -> 'g</w>' (Frequency: 5)
Iteration 9: Merged ('d', 'o') -> 'do' (Frequency: 4)
Iteration 10: Merged ('b', 'o') -> 'bo' (Frequency: 3)
Iteration 11: Merged ('bo', 'y') -> 'boy' (Frequency: 3)
Iteration 12: Merged ('g', 's</w>') -> 'gs</w>' (Frequency: 3)
Iteration 13: Merged ('c', 'a') -> 'ca' (Frequency: 3)
Iteration 14: Merged ('ca', 't') -> 'cat' (Frequency: 3)
Iteration 15: Merged ('i', 'n') -> 'in' (Frequency: 3)
Iteration 16: Merged ('in', 'g</w>') -> 'ing</w>' (Frequency:

In [2]:
import re
import collections
import numpy as np

def preprocess_sentence(sentence):
    """Applies all preprocessing rules to a single sentence."""
    s = re.sub(r'https?://\S+', 'URL', sentence)
    s = re.sub(r'[\$#]?\d+', 'NUMBER', s)
    s = re.sub(r'[!,.?:]', ' PUNCT ', s)
    tokens = s.lower().split()
    return tokens

def train_naive_bayes(data, k=0.3, verbose=False):
    """
    Trains the Naive Bayes classifier and returns a model with all probabilities.
    """
    if verbose:
        print("--- 1. Preprocessing Training Data ---")
        for sentence, label in data:
            print(f"Original: '{sentence}'")
            print(f"Processed: {preprocess_sentence(sentence)}\n")

    class_doc_counts = collections.defaultdict(int)
    feature_counts = { 'Inform': collections.defaultdict(int), 'Promo': collections.defaultdict(int), 'Reminder': collections.defaultdict(int) }
    bigram_counts = { 'Inform': collections.defaultdict(int), 'Promo': collections.defaultdict(int), 'Reminder': collections.defaultdict(int) }
    N_class = collections.defaultdict(int)
    vocab_bigram = set()

    for sentence, label in data:
        class_doc_counts[label] += 1
        tokens = preprocess_sentence(sentence)
        if 'URL' in tokens: feature_counts[label]['has_URL'] += 1
        if 'NUMBER' in tokens: feature_counts[label]['has_NUMBER'] += 1
        if 'PUNCT' in tokens: feature_counts[label]['has_PUNCT'] += 1
        bigrams = list(zip(tokens, tokens[1:]))
        for bigram in bigrams:
            bigram_counts[label][bigram] += 1
            vocab_bigram.add(bigram)
        N_class[label] += len(bigrams)
            
    total_docs = sum(class_doc_counts.values())
    labels = list(class_doc_counts.keys())
    
    priors = {label: count / total_docs for label, count in class_doc_counts.items()}
    
    feature_probs = {l: {} for l in labels}
    for label in labels:
        for feature in ['has_URL', 'has_NUMBER', 'has_PUNCT']:
            feature_probs[label][feature] = feature_counts[label][feature] / class_doc_counts[label]

    V_size = len(vocab_bigram)
    bigram_probs = {l: {} for l in labels}
    for label in labels:
        denominator = N_class[label] + k * V_size
        for bigram in vocab_bigram:
            numerator = bigram_counts[label].get(bigram, 0) + k
            bigram_probs[label][bigram] = numerator / denominator

    model = { 'priors': priors, 'feature_probs': feature_probs, 'bigram_probs': bigram_probs, 'labels': labels, 'bigram_vocab_size': V_size, 'N_class': N_class }
    
    if verbose:
        print_model_details(model, data[-1][0]) # Using last sentence's bigrams as example
        
    return model

def print_model_details(model, example_sentence):
    """Prints the key parameters of the trained model."""
    print("\n--- 2. Trained Model Parameters ---")
    print("\n[A] Class Priors P(Class):")
    for label, prior in model['priors'].items():
        print(f"{label:<10}: {prior:.4f}")

    print("\n[B] Specific Feature Probabilities P(Feature=1 | Class):")
    header = f"{'Feature':<15}" + "".join([f"{label:<10}" for label in model['labels']])
    print(header)
    print("-" * len(header))
    for feature in ['has_URL', 'has_NUMBER', 'has_PUNCT']:
        row = f"{feature:<15}"
        for label in model['labels']:
            row += f"{model['feature_probs'][label][feature]:<10.4f}"
        print(row)

    print("\n[C] Bigram Smoothing Parameters:")
    print(f"Total Unique Bigrams |V_bigram|: {model['bigram_vocab_size']}")
    print("Total Bigrams per Class (N_class):")
    for label, count in model['N_class'].items():
        print(f"{label:<10}: {count}")

    print("\n[D] Example Bigram Probabilities (for relevant bigrams):")
    test_bigrams_list = list(zip(preprocess_sentence(example_sentence), preprocess_sentence(example_sentence)[1:]))
    header = f"{'Bigram':<25}" + "".join([f"P(bigram|{label[:4]})" for label in model['labels']])
    print(header)
    print("-" * len(header))
    for bigram in test_bigrams_list:
        row = f"{str(bigram):<25}"
        for label in model['labels']:
            prob = model['bigram_probs'][label].get(bigram, "N/A")
            if isinstance(prob, float):
                row += f"{prob:<18.4f}"
            else:
                row += f"{'N/A':<18}"
        print(row)


def predict(sentence, model, verbose=False):
    """Predicts the label for a new sentence with verbose output."""
    labels = model['labels']
    tokens = preprocess_sentence(sentence)
    
    if verbose:
        print("\n--- 3. Prediction Phase ---")
        print(f"Test Sentence: '{sentence}'")
        print(f"Processed Tokens: {tokens}\n")

    has_URL = 1 if 'URL' in tokens else 0
    has_NUMBER = 1 if 'NUMBER' in tokens else 0
    has_PUNCT = 1 if 'PUNCT' in tokens else 0
    test_bigrams = list(zip(tokens, tokens[1:]))
    
    scores = {}
    for label in labels:
        if verbose: print(f"--- Calculating Score for Class: {label} ---")
        
        log_score = np.log(model['priors'][label])
        if verbose: print(f"  Log Prior P({label}): {log_score:.4f}")
        
        epsilon = 1e-9
        
        prob_url = model['feature_probs'][label]['has_URL']
        log_prob = np.log(prob_url + epsilon) if has_URL else np.log(1 - prob_url + epsilon)
        log_score += log_prob
        if verbose: print(f"  + Log P(has_URL={has_URL} | {label}): {log_prob:.4f}")
        
        prob_num = model['feature_probs'][label]['has_NUMBER']
        log_prob = np.log(prob_num + epsilon) if has_NUMBER else np.log(1 - prob_num + epsilon)
        log_score += log_prob
        if verbose: print(f"  + Log P(has_NUMBER={has_NUMBER} | {label}): {log_prob:.4f}")

        prob_punct = model['feature_probs'][label]['has_PUNCT']
        log_prob = np.log(prob_punct + epsilon) if has_PUNCT else np.log(1 - prob_punct + epsilon)
        log_score += log_prob
        if verbose: print(f"  + Log P(has_PUNCT={has_PUNCT} | {label}): {log_prob:.4f}")
        
        sum_log_bigram_probs = 0
        for bigram in test_bigrams:
            if bigram in model['bigram_probs'][label]:
                sum_log_bigram_probs += np.log(model['bigram_probs'][label][bigram])
        
        log_score += sum_log_bigram_probs
        if verbose: 
            print(f"  + Sum of Log Bigram Probs: {sum_log_bigram_probs:.4f}")
            print(f"  -------------------------------------")
            print(f"  Total Log Score for {label}: {log_score:.4f}\n")

        scores[label] = log_score
        
    return max(scores, key=scores.get), scores


# --- Main Execution ---
if __name__ == "__main__":
    training_data = [
        ("Check out https://example.com for more info!", "Inform"),
        ("Order 3 items, get 1 free! Limited offer!!!", "Promo"),
        ("Your package #12345 will arrive tomorrow.", "Inform"),
        ("Win $1000 now, visit http://winbig.com!!!", "Promo"),
        ("Meeting at 3pm, don't forget to bring the files.", "Reminder"),
        ("Exclusive deal for you: buy 2, get 1 free!!!", "Promo"),
        ("Download the report from https://reports.com.", "Inform"),
        ("The meeting is starting in 10 minutes.", "Reminder"),
        ("Reminder: submit your timesheet by 5pm today.", "Reminder")
    ]
    
    test_sentence = "You will get an exclusive offer in the meeting!"
    
    # Train the model with verbose=True to print training details
    nb_model = train_naive_bayes(training_data, k=0.3, verbose=True)
    
    # Make prediction with verbose=True to print prediction steps
    prediction, scores = predict(test_sentence, nb_model, verbose=True)
    
    print("\n--- 4. Final Result ---")
    print(f"Sentence to classify: '{test_sentence}'\n")
    print("--- Final Log Scores ---")
    for label, score in scores.items():
        print(f"{label:<10}: {score:.4f}")
    
    print("\n--------------------")
    print(f"Predicted Label: {prediction}")
    print("--------------------")

--- 1. Preprocessing Training Data ---
Original: 'Check out https://example.com for more info!'
Processed: ['check', 'out', 'url', 'for', 'more', 'info', 'punct']

Original: 'Order 3 items, get 1 free! Limited offer!!!'
Processed: ['order', 'number', 'items', 'punct', 'get', 'number', 'free', 'punct', 'limited', 'offer', 'punct', 'punct', 'punct']

Original: 'Your package #12345 will arrive tomorrow.'
Processed: ['your', 'package', 'number', 'will', 'arrive', 'tomorrow', 'punct']

Original: 'Win $1000 now, visit http://winbig.com!!!'
Processed: ['win', 'number', 'now', 'punct', 'visit', 'url']

Original: 'Meeting at 3pm, don't forget to bring the files.'
Processed: ['meeting', 'at', 'numberpm', 'punct', "don't", 'forget', 'to', 'bring', 'the', 'files', 'punct']

Original: 'Exclusive deal for you: buy 2, get 1 free!!!'
Processed: ['exclusive', 'deal', 'for', 'you', 'punct', 'buy', 'number', 'punct', 'get', 'number', 'free', 'punct', 'punct', 'punct']

Original: 'Download the report from